In [8]:
import ifcopenshell
import os
import csv
import unicodedata

# === Percorsi ===
source_folder = r"C:\Users\filippide.beltrame\OneDrive - Gruppo Carron\Desktop\from ifc to json\ifc\ifc"
output_csv_path = r"C:\Users\filippide.beltrame\OneDrive - Gruppo Carron\Desktop\abachi\python\abaco.csv"

# === Carica il file IFC (primo trovato nella cartella) ===
ifc_files = [f for f in os.listdir(source_folder) if f.lower().endswith(".ifc")]
if not ifc_files:
    raise FileNotFoundError("❌ Nessun file IFC trovato nella cartella.")
ifc_path = os.path.join(source_folder, ifc_files[0])
print(f"✅ File IFC trovato: {ifc_path}")
ifc_file = ifcopenshell.open(ifc_path)

# === Estrazione proprietà ===
elements = ifc_file.by_type("IfcProduct")
all_data = []
all_keys = set()

for el in elements:
    row = {}

    # Info base
    row["IFCCLASS"] = el.is_a()
    row["NAME"] = el.Name if el.Name else ""
    row["TAG"] = getattr(el, "Tag", "")
    row["TYPENAME"] = getattr(el, "ObjectType", "")

    # Livello
    for rel in getattr(el, "ContainedInStructure", []):
        if hasattr(rel, "RelatingStructure"):
            row["LEVEL"] = rel.RelatingStructure.Name

    # Proprietà personalizzate
    if hasattr(el, "IsDefinedBy"):
        for rel in el.IsDefinedBy:
            if hasattr(rel, "RelatingPropertyDefinition") and hasattr(rel.RelatingPropertyDefinition, "HasProperties"):
                for prop in rel.RelatingPropertyDefinition.HasProperties:
                    name = unicodedata.normalize('NFKD', str(prop.Name)).upper().replace(" ", "").replace("_", "")
                    try:
                        val = prop.NominalValue.wrappedValue
                    except:
                        val = getattr(prop, "NominalValue", None)
                    row[name] = val

    # Quantità (area, volume, lunghezza) lasciate come sono
    if hasattr(el, "IsDefinedBy"):
        for rel in el.IsDefinedBy:
            definition = getattr(rel, "RelatingPropertyDefinition", None)
            if definition and definition.is_a("IfcElementQuantity"):
                for qty in getattr(definition, "Quantities", []):
                    name = unicodedata.normalize('NFKD', str(qty.Name)).upper().replace(" ", "").replace("_", "")
                    for attr in ["AreaValue", "VolumeValue", "LengthValue"]:
                        if hasattr(qty, attr):
                            val = getattr(qty, attr)
                            if val:
                                row[name] = val

    # === Materiale
    if hasattr(el, "HasAssociations"):
        for assoc in el.HasAssociations:
            if assoc.is_a("IfcRelAssociatesMaterial"):
                mat = assoc.RelatingMaterial
                try:
                    if mat.is_a("IfcMaterial"):
                        row["MATERIALE"] = mat.Name
                    elif hasattr(mat, "ForLayerSet") and hasattr(mat.ForLayerSet, "MaterialLayers"):
                        layers = [l.Material.Name for l in mat.ForLayerSet.MaterialLayers if hasattr(l, "Material")]
                        row["MATERIALE"] = ", ".join(layers)
                except:
                    pass

    all_data.append(row)
    all_keys.update(row.keys())

# === Mostra intestazioni disponibili
print("\n🔎 Colonne disponibili:")
column_list = sorted(all_keys)
for col in column_list:
    print(f"- {col}")

# === Input colonne (una riga, separate da virgola)
print("\n✍️ Inserisci le colonne da esportare, separate da virgola:")
raw_input = input("Colonne: ")
selected = [col.strip().upper().replace(" ", "").replace("_", "") for col in raw_input.split(",") if col.strip()]

if not selected:
    raise ValueError("❌ Nessuna colonna selezionata.")

# === Esporta CSV
with open(output_csv_path, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=selected, delimiter=';')
    writer.writeheader()
    for row in all_data:
        filtered_row = {k: row.get(k, "") for k in selected}
        writer.writerow(filtered_row)

print(f"\n✅ Esportazione completata: {output_csv_path}")


✅ File IFC trovato: C:\Users\filippide.beltrame\OneDrive - Gruppo Carron\Desktop\from ifc to json\ifc\ifc\20230328_769-CR-PE-ARC_M3.ifc

🔎 Colonne disponibili:
- AREA
- CRTAGCODE
- CRTAGCODEDE
- CRWBSCODE
- CRWBSFASE
- CRWBSLIVELLO
- ELETTROSERRATURA
- IFCCLASS
- LEVEL
- LUNGHEZZA
- MAGNETE
- MANIGLIONEANTIPANICO
- MATERIALE
- NAME
- NOMEDELTIPO
- RESISTENZAALFUOCO
- TAG
- TYPENAME
- VOLUME

✍️ Inserisci le colonne da esportare, separate da virgola:

✅ Esportazione completata: C:\Users\filippide.beltrame\OneDrive - Gruppo Carron\Desktop\abachi\python\abaco.csv
